# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298359784894                   -0.85    4.8         
  2   -8.300182953412       -2.74       -1.26    1.2    174ms
  3   -8.300427829600       -3.61       -1.89    3.5    320ms
  4   -8.300462349684       -4.46       -2.77    1.6    157ms
  5   -8.300464345994       -5.70       -3.09    3.0    220ms
  6   -8.300464501152       -6.81       -3.28    9.9    344ms
  7   -8.300464573988       -7.14       -3.43    1.2    185ms
  8   -8.300464608127       -7.47       -3.57    2.1    198ms
  9   -8.300464634798       -7.57       -3.78    1.9    196ms
 10   -8.300464638507       -8.43       -3.88    2.4    248ms
 11   -8.300464643246       -8.32       -4.15    3.1    297ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.63663702279                   -0.71    4.5         
  2   -16.64096220308       -2.36       -1.15    1.0    352ms
  3   -16.67838349634       -1.43       -1.77    2.1    475ms
  4   -16.67923826482       -3.07       -2.24    4.6    564ms
  5   -16.67927822252       -4.40       -2.63    3.2    495ms
  6   -16.67928600456       -5.11       -3.23    2.2    411ms
  7   -16.67928619205       -6.73       -3.44    4.1    583ms
  8   -16.67928621996       -7.55       -4.12    1.0    359ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32526075676                   -0.56    7.1         
  2   -33.33245283803       -2.14       -1.00    1.0    1.35s
  3   -33.33412651934       -2.78       -1.73    4.8    1.90s
  4   -33.33428423221       -3.80       -2.65    2.0    1.37s
  5   -33.33557400682       -2.89       -2.41    5.5    2.33s
  6   -33.33694269450       -2.86       -2.53    7.8    1.86s
  7   -33.33694318912       -6.31       -2.56    1.4    1.31s
  8   -33.33687960625   +   -4.20       -2.42    1.9    1.38s
  9   -33.33686755984   +   -4.92       -2.39    1.0    1.26s
 10   -33.33687646174       -5.05       -2.42    1.0    1.30s
 11   -33.33690776399       -4.50       -2.56    1.0    1.30s
 12   -33.33690840049       -6.20      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298471816809                   -0.85    5.1         
  2   -8.300250994434       -2.75       -1.59    1.0    121ms
  3   -8.300410726852       -3.80       -2.43    2.5    165ms
  4   -8.300341269697   +   -4.16       -2.20    7.5    306ms
  5   -8.300464093046       -3.91       -3.51    1.0    151ms
  6   -8.300464563426       -6.33       -3.77    3.8    284ms
  7   -8.300464638473       -7.12       -4.17    1.2    154ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32491742814                   -0.56    6.4         
  2   -33.30695538238   +   -1.75       -1.26    1.6    1.15s
  3   -8.771039673142   +    1.39       -0.37    7.0    2.61s
  4   -33.32308655233        1.39       -1.71    5.9    2.48s
  5   -33.12569608630   +   -0.70       -1.25    4.5    2.59s
  6   -30.99338864186   +    0.33       -0.88    4.4    1.99s
  7   -33.32661672443        0.37       -1.96    4.5    1.99s
  8   -33.33069282750       -2.39       -2.03    2.0    1.44s
  9   -33.33457565721       -2.41       -2.11    2.4    1.55s
 10   -33.33625191167       -2.78       -2.61    1.8    1.38s
 11   -33.33689458234       -3.19       -3.05    3.0    1.67s
 12   -33.33688600357   +   -5.07      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.